In [1]:
#| echo: false
from utils import build_buttons
from importlib import reload
import utils
reload(utils)
utils.build_buttons(link= 'xpath', 
                    github= 'https://github.com/yinleon/inspect-element/blob/main/xpath.ipynb',
                    citation= True)


<a type="button" class="btn btn-outline-primary btn-sm" target="_blank" href="https://inspectelement.org/xpath">📖 Read online</a>
<a type="button" class="btn btn-outline-primary btn-sm" target="_blank" href="https://colab.research.google.com/github/yinleon/inspect-element/blob/main/xpath.ipynb">🖥️  Interactive version</a>
<a type="button" class="btn btn-outline-primary btn-sm" target="_blank" href="https://github.com/yinleon/inspect-element/blob/main/xpath.ipynb">⚙️ GitHub</a>
<a type="button" class="btn btn-outline-primary btn-sm" target="_self" href="#citation">🏛 Citation</a>
<br>


[XPath](https://en.wikipedia.org/wiki/XPath) is an expression language used to parse and navigate [XML](https://en.wikipedia.org/wiki/XML)-formatted documents such as HTML.

XPath is useful for web scraping and can be used in place of the popular [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/) Python package. 

A key advantage of XPath is its universality. The same selectors will work across browsers and web scraping tools, which smoothes the lines between inspecting live websites, writing code to pull data from HTML pages, and identifying specific web elements to interact with using [browser automation](/browser_automation.html) tools.

This section will teach you how to write simple, precise, and generalizable XPath expressions to parse and manipulate web pages.

# Introduction to XPath

We're going to identify all the recent article titles and links from NPR.
In your browser and go to our example website: https://text.npr.org/

Next, open the dev tools by right-clicking anywhere and selecting "Inspect" (or however else).

Select any element in the "Elements" tab and copy the XPath:

<figure>
    <img src='assets/xpath_console.png' width=75%>
    <figcaption align = "left" style="font-size:80%;"> 
    How to copy the xpath of an element in Dev Tools.
    </figcaption>
</figure>

The element we're selecting is an `<a>` tag with a link and a title that looks like this:

```
<a class="topic-title" href="/nx-s1-5035272">What is in Project 2025? </a>
```

The resulting XPath that we copied looks like this:

```
/html/body/main/div/ul/li[1]/a
```

### What is XPath?

XPath records hierarchically across a cascade of HTML tags, with the last tag denoting the destination.

It designates where an element lives in an HTML document (as if you were honing in on a street address from the center of the earth).

The example above is long and specific to one element on the page (BAD!). It is called an _absolute XPath_. 

Absolute XPath provides directions to a _specifc_ destination (for example the Shake Shack in Madison Square). However, there's another form of XPath called a _relative XPath_ that can provide directions to _every_ Shake Shack.

With a little practice XPath can be both precise and generalizable, providing an elegant way to locate and select elements from web pages.

Here is the other extreme: short and generic (ALSO BAD!!). 

```
//a
```

This _relative XPath_ syntax yields the target element mixed with every other element on the page with an `<a>` tag. Following the Shake Shack analogy, this XPath represents directions to every restaurant on Earth.

You'll notice the "//" before the `<a>` tag, which denotes a search _anywhere_ on the page. 

### Identifying the optimal XPath in the browser

A key strength of XPath is that you can identify and refine them _in browser_, and use the same XPath in different frameworks to make web parsing a breeze. 

The Goldie Locks approach is not about specifying the exact route (absolute XPath), but rather identifying the destinguishing attributes of the destination (a precise relative XPath).

Let's jump back to the live [NPR website](https://text.npr.org/):

1. In Dev Tools, switch over to the "console" tab. This allows us to execute JavaScript on the page.

2. We'll use the `$x()` [function](https://developer.chrome.com/docs/devtools/console/utilities#xpath-function) to select elements on the page by xpath ("x" for **x**path). This is unique to devtools and not a standard JavaScript function.

As a start, type a HTML tag such as a "header", "a", "div":
```
$x('//a')
```

XPath offers an easy way specify attributes and other distinguishable features. 

You just add an "@" sign before the attribute name. This allows you to denote specific attribute values `//a[@href="/nx-s1-5035272"]` or simply the presence of an attribute `//a[@href]`. 

Here's a workflow for honing in to the optimal XPath: 

- Start a XPath with a HTML tag with closed brackets: `//a[ ... ]`
- copy + paste attributes from a live element (for example `<a class="topic-title" href="/nx-s1-5035272">`)
- Add "@" before each attribute, ending up with: `//a[@class="topic-title" and @href="/nx-s1-5035272"]`.

From there, you can remove overly-specific attributes. In the case above, the `class` is unique enough to isolate news articles.

```
$x('//a[@class="topic-title"]')
```

When available, identify elements using accessibility features (`aria`) or `data` attributes. These are less subject to change and are easy to read.

## Text Matching
XPath also allows for text-matching. 

Here's how you can match for a link on the page with text mentioning "2025"
```
$x('//a[@href and contains(text(),"2025")]')
```
To sanity check your results, you can expand the resulting list and click any of the elements. This will highlight the element on the page and shoot you back to the Dev Tools "Elements" tab to view the element.


<video width=100% controls loop>
    <source src="assets/click_xpath.mp4" type=video/mp4>
</video>

In [ ]:
/html/body/main/div/ul/li[1]/a

# Web Parsing with XPath 

## Static HTML

With the correct xpath in hand, we can automate this parsing in Python using the `lxml` package.

In [4]:
!pip install lxml beautifulsoup4

In [2]:
import requests

Let's visit the website and retrieve the static HTML from the page.

In [3]:
url = "https://text.npr.org/"
resp = requests.get(url)

If you're familar with Beautiful Soup, we're going to be doing the same steps as below:

In [30]:
from bs4 import BeautifulSoup

# read the webpage as bs4
soup = BeautifulSoup(resp.text)

# select all the "a" tags with the specified class.
articles = soup.find_all("a", {"class": "topic-title"})

# iterate through each headline and grab the title and link of each story.
data = []
for elm in articles:
    link = elm.get('href')
    link = f"https://npr.org{link}"
    title = elm.text
    row = {'link' : link, 'title': title}
    data.append(row)
    
data[:5]

[{'link': 'https://npr.org/nx-s1-5458512',
  'title': 'How good was the forecast? Texas officials and the National Weather Service disagree'},
 {'link': 'https://npr.org/nx-s1-5458514',
  'title': 'Texas officials race to find survivors after devastating floods '},
 {'link': 'https://npr.org/nx-s1-5457278',
  'title': 'At least 78 dead and dozens missing after catastrophic Texas flooding '},
 {'link': 'https://npr.org/nx-s1-5454890',
  'title': '4 things to know about the vaccine ingredient thimerosal'},
 {'link': 'https://npr.org/g-s1-75874',
  'title': 'Knives, bullets and thieves: the quest for food in Gaza'}]

Here we'll read the HTML into the tree to parse with XPath. Note: some frameworks prefer the `.//` relative syntax (lxml) compared to `//` (Playwright).

In [32]:
from lxml import etree

# read the webpage as lxml
tree = etree.HTML(resp.text)

# select all the "a" tags with the specified class.
xpath_articles = './/a[@class="topic-title"]'
elements = tree.findall(xpath_articles)

# iterate through each headline and grab the title and link of each story.
data = []
for elm in elements:
    link = elm.get('href')
    link = f"https://npr.org{link}"
    title = elm.text
    row = {'link' : link, 'title': title}
    data.append(row)
    
data[:5]

[{'link': 'https://npr.org/nx-s1-5458512',
  'title': 'How good was the forecast? Texas officials and the National Weather Service disagree'},
 {'link': 'https://npr.org/nx-s1-5458514',
  'title': 'Texas officials race to find survivors after devastating floods '},
 {'link': 'https://npr.org/nx-s1-5457278',
  'title': 'At least 78 dead and dozens missing after catastrophic Texas flooding '},
 {'link': 'https://npr.org/nx-s1-5454890',
  'title': '4 things to know about the vaccine ingredient thimerosal'},
 {'link': 'https://npr.org/g-s1-75874',
  'title': 'Knives, bullets and thieves: the quest for food in Gaza'}]

The outcome is the same between BeautifulSoup and lxml. However BeautifulSoup only works with static HTML you've saved or stored in memory.

XPath can be used to parse data in this format, but also can be used to work in browser automation frameworks.

## Browser Automation

The same xpath can be used for browser automation frameworks, such as Playwright.

In [10]:
# download software
!pip install playwright
!playwright install

In [34]:
from playwright.async_api import async_playwright

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

The exact function to use XPath is a little different across scraping tools ('locator' in the case of Playwright), but the XPath expressions stay the same. 

In [35]:
# go to the webpage,
await page.goto(url)

# we use xpath in the `locator` function to select all the articles.
xpath_article = '//a[@class="topic-title"]'
elms = await page.locator(xpath_article).all()

In [36]:
# iterate through each headline and grab the title and link of each story.
data = []
for elm in elms:
    title = await elm.text_content()
    link = await elm.get_attribute('href')
    link = f'https://npr.org{link}'
    row = {'link': link, 'title': title,}
    data.append(row)

data[:5]

[{'link': 'https://npr.org/nx-s1-5458512',
  'title': 'How good was the forecast? Texas officials and the National Weather Service disagree'},
 {'link': 'https://npr.org/nx-s1-5458514',
  'title': 'Texas officials race to find survivors after devastating floods '},
 {'link': 'https://npr.org/nx-s1-5457278',
  'title': 'At least 78 dead and dozens missing after catastrophic Texas flooding '},
 {'link': 'https://npr.org/nx-s1-5454890',
  'title': '4 things to know about the vaccine ingredient thimerosal'},
 {'link': 'https://npr.org/g-s1-75874',
  'title': 'Knives, bullets and thieves: the quest for food in Gaza'}]

In [37]:
# shut down the browser to free up resources.
await browser.close()

In [58]:
#| output: false
#| echo: false
import pandas as pd
x = pd.DataFrame([
    {'Platform': 'BeautifulSoup', 'Function' : '`find_all`', 'Example' : '`soup.find_all("a", {"class": "topic-title"})`'},
    {'Platform': 'Browser console', 'Function' : '`$x`', 'Example' : '`$x(\'//a[@class="topic-title"]\')`'},
    {'Platform': 'lxml', 'Function' : '`findall`', 'Example' : '`tree.findall(\'//a[@class="topic-title"]\')`'},
    {'Platform': 'Playwright', 'Function' : '`locator`', 'Example' : '`page.locator(\'//a[@class="topic-title"]\').all()`'},
]).to_markdown(index=False)
print(x)

| Platform        | Function   | Example                                           |
|:----------------|:-----------|:--------------------------------------------------|
| bs4             | `find_all` | `soup.find_all("a", {"class": "topic-title"})`    |
| Browser console | `$x`       | `$x('//a[@class="topic-title"]')`                 |
| lxml            | `findall`  | `tree.findall('//a[@class="topic-title"]')`       |
| Playwright      | `locator`  | `page.locator('//a[@class="topic-title"]').all()` |


To review, here is how we would find all the article elements across the frameworks we mentioned...

# Equivalent functions across frameworks

| Framework       | Function   | Example                                           |
|:----------------|:-----------|:--------------------------------------------------|
| bs4             | `find_all` | `soup.find_all("a", {"class": "topic-title"})`    |
| Browser console | `$x`       | `$x('//a[@class="topic-title"]')`                 |
| lxml            | `findall`  | `tree.findall('//a[@class="topic-title"]')`       |
| Playwright      | `locator`  | `page.locator('//a[@class="topic-title"]').all()` |

## When things get more complicated

We exhibited XPath's basic functionality on a simple static HTML page. It really shines when it comes to breaking down a complicated website